In [1]:
### from hpc1

In [7]:
import matplotlib.animation as animation
from matplotlib import rc
from IPython.display import HTML, Image
from ipywidgets import interact,Dropdown,IntSlider,FloatRangeSlider, FloatSlider, RadioButtons
rc('animation', html='html5')
import gc, argparse, sys, os, errno
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from beakerx import *
plt.style.use('ggplot')
import seaborn as sns
sns.set()
import h5py
import os
import scipy
import sklearn
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


/home/xupeng/.local/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['rc', 'random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


## 整个过程

In [ ]:
data_sources = tuple(
        Hdf5Source(
            'sample_'+s+'_padded_20160501.aligned.filled.cropped.hdf',
            datasets = {
                ArrayKeys.RAW: 'volumes/raw',
                ArrayKeys.GT_LABELS: 'volumes/labels/neuron_ids_notransparency',
                ArrayKeys.GT_MASK: 'volumes/labels/mask',
            }
        ) +
        Normalize() +
        RandomLocation()
        for s in ['A', 'B', 'C']
    )

    artifact_source = (
        Hdf5Source(
            'sample_ABC_padded_20160501.defects.hdf',
            datasets = {
                ArrayKeys.RAW: 'defect_sections/raw',
                ArrayKeys.ALPHA_MASK: 'defect_sections/mask',
            }
        ) +
        RandomLocation(min_masked=0.05, mask_array_key=ArrayKeys.ALPHA_MASK) +
        Normalize() +
        IntensityAugment(0.9, 1.1, -0.1, 0.1, z_section_wise=True) +
        ElasticAugment([4,40,40], [0,2,2], [0,math.pi/2.0]) +
        SimpleAugment(transpose_only_xy=True)
    )

    snapshot_request = BatchRequest()
    snapshot_request.add_array_request(ArrayKeys.LOSS_GRADIENT, (56,56,56))

    batch_provider_tree = (
        data_sources +
        RandomProvider() +
        ExcludeLabels([8094], ignore_mask_erode=12) +
        ElasticAugment([4,40,40], [0,2,2], [0,math.pi/2.0], prob_slip=0.05,prob_shift=0.05,max_misalign=25) +
        SimpleAugment(transpose_only_xy=True) +
        GrowBoundary(steps=3, only_xy=True) +
        AddGtAffinities(affinity_neighborhood) +
        SplitAndRenumberSegmentationLabels() +
        IntensityAugment(0.9, 1.1, -0.1, 0.1, z_section_wise=True) +
        DefectAugment(
            prob_missing=0.03,
            prob_low_contrast=0.01,
            prob_artifact=0.03,
            artifact_source=artifact_source,
            contrast_scale=0.1) +
        ZeroOutConstSections() +
        IntensityScaleShift(2,-1) +
        BalanceAffinityLabels() +
        PreCache(
            cache_size=10,
            num_workers=5) +
        Train(solver_parameters, use_gpu=0) +
        Snapshot(every=10, output_filename='batch_{id}.hdf', additional_request=snapshot_request)
    )

    n = 10
    print("Training for", n, "iterations")

    with build(batch_provider_tree) as minibatch_maker:
        for i in range(n):
            minibatch_maker.request_batch(request)

    print("Finished")

# API
## DefectAugment
class gunpowder.DefectAugment(intensities, prob_missing=0.05, prob_low_contrast=0.05, prob_artifact=0.0, prob_deform=0.0, contrast_scale=0.1, artifact_source=None, artifacts=None, artifacts_mask=None, deformation_strength=20, axis=0)[source]

Augment intensity arrays section-wise with artifacts like missing sections, low-contrast sections, by blending in artifacts drawn from a separate source, or by deforming a section.

- intensities (ArrayKey) – The key of the array of intensities to modify.
- prob_missing (float) –
- prob_low_contrast (float) –
- prob_artifact (float) –
- prob_deform (float) – Probabilities of having a missing section, low-contrast section, an artifact (see param artifact_source) or a deformed slice. The sum should not exceed 1. Values in missing sections will be set to 0.
contrast_scale (float, optional) – By how much to scale the intensities for a low-contrast section, used if prob_low_contrast > 0.
- (class (artifact_source) – BatchProvider, optional):A gunpowder batch provider that delivers intensities (via ArrayKey artifacts) and an alpha mask (via ArrayKey artifacts_mask), used if prob_artifact > 0.

- artifacts (ArrayKey, optional) – The key to query artifact_source for to get the intensities of the artifacts.
- artifacts_mask (ArrayKey, optional) – The key to query artifact_source for to get the alpha mask of the artifacts to blend them with intensities.
- deformation_strength (int, optional) – Strength of the slice deformation in voxels, used if prob_deform > 0. The deformation models a fold by shifting the section contents towards a randomly oriented line in the section. The line itself will be drawn with a value of 0.
- axis (int, optional) – Along which axis sections are cut.


## ElasticAugment
class gunpowder.ElasticAugment(control_point_spacing, jitter_sigma, rotation_interval, prob_slip=0, prob_shift=0, max_misalign=0, subsample=1)[source]
Elasticly deform a batch. Requests larger batches upstream to avoid data loss due to rotation and jitter.

- control_point_spacing (tuple of int) – Distance between control points for the elastic deformation, in voxels per dimension.
- jitter_sigma (tuple of float) – Standard deviation of control point jitter distribution, in voxels per dimension.
rotation_interval (tuple of two floats) – Interval to randomly sample rotation angles from (0, 2PI).
- prob_slip (float) – Probability of a section to “slip”, i.e., be independently moved in x-y.
- prob_shift (float) – Probability of a section and all following sections to move in x-y.
- max_misalign (int) – Maximal voxels to shift in x and y. Samples will be drawn uniformly. Used if prob_slip + prob_shift > 0.
- subsample (int) – Instead of creating an elastic transformation on the full resolution, create one subsampled by the given factor, and linearly interpolate to obtain the full resolution transformation. This can significantly speed up this node, at the expense of having visible piecewise linear deformations for large factors. Usually, a factor of 4 can savely by used without noticable changes. However, the default is 1 (i.e., no subsampling).

## IntensityAugment
class gunpowder.IntensityAugment(array, scale_min, scale_max, shift_min, shift_max, z_section_wise=False)[source]
Randomly scale and shift the values of an intensity array.

- array (ArrayKey) – The intensity array to modify.
- scale_min (float) –
- scale_max (float) –
- shift_min (float) –
- shift_max (float) –
The min and max of the uniformly randomly drawn scaling and shifting values for the intensity augmentation. Intensities are changed as:
```
a = a.mean() + (a-a.mean())*scale + shift
```
- z_section_wise (bool) – Perform the augmentation z-section wise. Requires 3D arrays and assumes that z is the first dimension.

## SimpleAugment
class gunpowder.SimpleAugment(mirror_only=None, transpose_only=None)[source]
Randomly mirror and transpose all Arrays and Points in a batch.

Parameters:	
- mirror_only (list of int, optional) – If set, only mirror between the given axes. This is useful to exclude channels that have a set direction, like time.
- transpose_only (list of int, optional) – If set, only transpose between the given axes. This is useful to limit the transpose to axes with the same resolution or to exclude non-spatial dimensions.

In [3]:
#import gunpowder

# first test simple augmentation, try without installing gunpowder!

In [ ]:
class SimpleAugment(BatchFilter):
    '''Randomly mirror and transpose all :class:`Arrays<Array>` and
    :class:`Points` in a batch.
    Args:
        mirror_only (``list`` of ``int``, optional):
            If set, only mirror between the given axes. This is useful to
            exclude channels that have a set direction, like time.
        transpose_only (``list`` of ``int``, optional):
            If set, only transpose between the given axes. This is useful to
            limit the transpose to axes with the same resolution or to exclude
            non-spatial dimensions.
    '''

    def __init__(self, mirror_only=None, transpose_only=None):

        self.mirror_only = mirror_only
        self.transpose_only = transpose_only
        self.mirror_mask = None
        self.dims = None
        self.transpose_dims = None

    def setup(self):

        self.dims = self.spec.get_total_roi().dims()

        if self.mirror_only is None:
            self.mirror_mask = [ True ]*self.dims
        else:
            self.mirror_mask = [ d in self.mirror_only for d in range(self.dims) ]

        if self.transpose_only is None:
            self.transpose_dims = list(range(self.dims))
        else:
            self.transpose_dims = self.transpose_only

    def prepare(self, request):

        self.total_roi = request.get_total_roi()

        self.mirror = [
            random.randint(0,1)
            if self.mirror_mask[d] else 0
            for d in range(self.dims)
        ]

        t = list(self.transpose_dims)
        random.shuffle(t)
        self.transpose = list(range(self.dims))
        for o, n in zip(self.transpose_dims, t):
            self.transpose[o] = n

        logger.debug("mirror = " + str(self.mirror))
        logger.debug("transpose = " + str(self.transpose))

        reverse_transpose = [0]*self.dims
        for d in range(self.dims):
            reverse_transpose[self.transpose[d]] = d

        logger.debug("downstream request = " + str(request))

        self.__transpose_request(request, reverse_transpose)
        self.__mirror_request(request, self.mirror)

        logger.debug("upstream request = " + str(request))

    def process(self, batch, request):

        mirror = tuple(
                slice(None, None, -1 if m else 1)
                for m in self.mirror
        )
        # arrays
        for (array_key, array) in batch.arrays.items():

            array.data = array.data[mirror]
            if self.transpose != (0,1,2):
                array.data = array.data.transpose(self.transpose)
        # points
        total_roi_offset = self.total_roi.get_offset()
        for (points_key, points) in batch.points.items():

            for loc_id, syn_point in points.data.items():
                # mirror
                location_in_total_offset = np.asarray(syn_point.location) - total_roi_offset
                syn_point.location = np.asarray([self.total_roi.get_end()[dim] - location_in_total_offset[dim]
                                                 if m else syn_point.location[dim] for dim, m in enumerate(self.mirror)])
                # transpose
                if self.transpose != (0, 1, 2):
                    syn_point.location = np.asarray([syn_point.location[self.transpose[d]] for d in range(self.dims)])

                # due to the mirroring, points at the lower boundary of the ROI
                # could fall on the upper one, which excludes them from the ROI
                if not points.spec.roi.contains(syn_point.location):
                    del points.data[loc_id]

        # arrays & points
        for collection_type in [batch.arrays, batch.points]:
            for (type, collector) in collection_type.items():
                logger.debug("total ROI: %s"%self.total_roi)
                logger.debug("upstream %s ROI: %s"%(type, collector.spec.roi))
                self.__mirror_roi(collector.spec.roi, self.total_roi, self.mirror)
                logger.debug("mirrored %s ROI: %s"%(type,collector.spec.roi))
                self.__transpose_roi(collector.spec.roi, self.transpose)
                logger.debug("transposed %s ROI: %s"%(type,collector.spec.roi))


    def __mirror_request(self, request, mirror):

        for key, spec in request.items():
            self.__mirror_roi(spec.roi, self.total_roi, mirror)

    def __transpose_request(self, request, transpose):

        for key, spec in request.items():
            self.__transpose_roi(spec.roi, transpose)

    def __mirror_roi(self, roi, total_roi, mirror):

        total_roi_offset = total_roi.get_offset()
        total_roi_shape = total_roi.get_shape()

        roi_offset = roi.get_offset()
        roi_shape = roi.get_shape()

        roi_in_total_offset = roi_offset - total_roi_offset
        end_of_roi_in_total = roi_in_total_offset + roi_shape
        roi_in_total_offset_mirrored = total_roi_shape - end_of_roi_in_total
        roi_offset = Coordinate(
                total_roi_offset[d] + roi_in_total_offset_mirrored[d] if mirror[d] else roi_offset[d]
                for d in range(self.dims)
        )

        roi.set_offset(roi_offset)

    def __transpose_roi(self, roi, transpose):

        offset = roi.get_offset()
        shape = roi.get_shape()
        offset = tuple(offset[transpose[d]] for d in range(self.dims))
        shape = tuple(shape[transpose[d]] for d in range(self.dims))
        roi.set_offset(offset)
        roi.set_shape(shape)

## try mirror!

In [5]:
def setup(mirror_only=None,transpose_only=None,dims=3):
    if mirror_only is None:
        mirror_mask = [ True ]*dims
    else:
        mirror_mask = [ d in self.mirror_only for d in range(dims) ]

    if transpose_only is None:
        transpose_dims = list(range(dims))
    else:
        transpose_dims = transpose_only
    return mirror_mask,transpose_dims

In [11]:
mirror_mask,transpose_dims = setup(mirror_only=None,transpose_only=None,dims=3)
print (mirror_mask,transpose_dims)

[True, True, True] [0, 1, 2]


In [24]:
mirror_mask

[True, True, True]

In [27]:
dims=3
#问题np.random.randint(0,1)只能产生0？！
# random.randint generate random axis?
[np.random.randint(0,1) if mirror_mask[d] else 0 for d in range(dims)]

[0, 0, 0]

In [ ]:
def get_total_roi(self):
    '''Get the union of all the ROIs.'''

    total_roi = None
    for specs_type in [self.array_specs, self.points_specs]:
        for (_, spec) in specs_type.items():
            if total_roi is None:
                total_roi = spec.roi
            else:
                total_roi = total_roi.union(spec.roi)
    return total_roi

In [12]:
tt = {}
tt is not None

True

In [41]:
list(range(dims))

[0, 1, 2]

In [11]:
def get_total_roi():
    skip

In [ ]:
def prepare(dims, request,transpose_dims):
    #self.total_roi = request.get_total_roi()

    mirror = [
        random.randint(0,1)
        if self.mirror_mask[d] else 0
        for d in range(dims)
    ]   #只能产生0？

    t = list(transpose_dims)
    random.shuffle(t)
    transpose = list(range(dims))
    for o, n in zip(transpose_dims, t):
        transpose[o] = n

    logger.debug("mirror = " + str(mirror))
    logger.debug("transpose = " + str(transpose))

    reverse_transpose = [0]*dims
    for d in range(dims):
        reverse_transpose[transpose[d]] = d

    logger.debug("downstream request = " + str(request))

    self.__transpose_request(request, reverse_transpose)
    self.__mirror_request(request, self.mirror)

    logger.debug("upstream request = " + str(request))

## simple augment can be achieved by EM-segLib/em_segLib/transform.py!
in this repo, only transpose_only_xy=True is used!

# IntensityAugment
class gunpowder.IntensityAugment(array, scale_min, scale_max, shift_min, shift_max, z_section_wise=False)[source]
Randomly scale and shift the values of an intensity array.

- array (ArrayKey) – The intensity array to modify.
- scale_min (float) –
- scale_max (float) –
- shift_min (float) –
- shift_max (float) –
The min and max of the uniformly randomly drawn scaling and shifting values for the intensity augmentation. Intensities are changed as:
```
a = a.mean() + (a-a.mean())*scale + shift
```
- z_section_wise (bool) – Perform the augmentation z-section wise. Requires 3D arrays and assumes that z is the first dimension.

## Randomly scale and shift the values of an intensity array，
- 可以对z方向做逐层的的augmentation，这时要求第一个dimension是z
- 要求array是normalize过得，0到1之内的
- 需要制定scale和shift分别的最大最小值，然后通过
```
np.random.uniform(low=self.shift_min, high=self.shift_max))
```
获得一个scale和shift的值。
- 每个array进行如下操作：a.mean() + (a-a.mean())*scale + shift
- 对于返回的array，大于1和小于0的分别赋值为1和0

## similar implementation!
- scale is similar to DataProvider/python/data_augmentation.py class GreyAugment(DataAugment)!
- DataProvider/python/data_augmentation.py class GreyAugment(DataAugment)的方案**可能更好**：
```
img *= 1 + (np.random.rand() - 0.5)*self.CONTRAST_FACTOR
img += (np.random.rand() - 0.5)*self.BRIGHTNESS_FACTOR
img = np.clip(img, 0, 1)
img **= 2.0**(np.random.rand()*2 - 1)
```
- 该方案是从ELEKTRONN (http://elektronn.org/).借鉴来的，ELEKTRONN似乎是一个专门处理2D/3D large scale image的库，也可以用来做segmentation

# ElasticAugment

In [11]:
from scipy.ndimage.interpolation import map_coordinates, zoom
def create_elastic_transformation(shape, control_point_spacing = 100, jitter_sigma = 10.0, subsample = 1):

    dims = len(shape)
    subsample_shape = tuple(max(1,int(s/subsample)) for s in shape)

    try:
        spacing = tuple((d for d in control_point_spacing))
    except:
        spacing = (control_point_spacing,)*dims
    try:
        sigmas = [ s for s in jitter_sigma ]
    except:
        sigmas = [jitter_sigma]*dims

    control_points = tuple(
            max(1,int(round(float(shape[d])/spacing[d])))
            for d in range(len(shape))
    )

    # jitter control points
    control_point_offsets = np.zeros((dims,) + control_points, dtype=np.float32)
    for d in range(dims):
        if sigmas[d] > 0:
            control_point_offsets[d] = np.random.normal(scale=sigmas[d], size=control_points)
    print (control_point_offsets.shape)
    return upscale_transformation(control_point_offsets, subsample_shape, interpolate_order=3)
def upscale_transformation(transformation, output_shape, interpolate_order=1):

    input_shape = transformation.shape[1:]

    # print("Upscaling control points")
    # print("\tfrom               : " + str(input_shape))
    # print("\tto                 : " + str(output_shape))
    # print("\tinterpolation order: " + str(interpolate_order))

    dims = len(output_shape)
    scale = tuple(float(s)/c for s,c in zip(output_shape, input_shape))

    start = time.time()
    scaled = np.zeros((dims,)+output_shape, dtype=np.float32)
    for d in range(dims):
        zoom(transformation[d], zoom=scale, output=scaled[d], order=interpolate_order)
    # print("\tupsampled in " + str(time.time() - start) + "s")

    return scaled


In [13]:
def create_rotation_transformation(shape, angle, subsample=1):

    dims = len(shape)
    subsample_shape = tuple(max(1,int(s/subsample)) for s in shape)
    control_points = (2,)*dims

    # map control points to world coordinates
    control_point_scaling_factor = tuple(float(s-1) for s in shape)

    # rotate control points
    center = np.array([0.5*(d-1) for d in shape])

    # print("Creating rotation transformation with:")
    # print("\tangle : " + str(angle))
    # print("\tcenter: " + str(center))

    control_point_offsets = np.zeros((dims,) + control_points, dtype=np.float32)
    for control_point in np.ndindex(control_points):

        point = np.array(control_point)*control_point_scaling_factor
        center_offset = np.array([p-c for c,p in zip(center, point)], dtype=np.float32)
        rotated_offset = np.array(center_offset)
        rotated_offset[-2:] = rotate(center_offset[-2:], angle)
        displacement = rotated_offset - center_offset
        control_point_offsets[(slice(None),) + control_point] += displacement

    return upscale_transformation(control_point_offsets, subsample_shape)

In [12]:
create_elastic_transformation(
        (10,100,100),
        control_point_spacing = 100,
        #num_control_points = [3,10,10],
        jitter_sigma = [0.3, 10, 10]).shape

(3, 1, 1, 1)


(3, 10, 100, 100)

In [ ]:
def __misalign(self, transformation):

    num_sections = transformation[0].shape[0]

    shifts = [Coordinate((0,0,0))]*num_sections
    for z in range(num_sections):

        r = random.random()

        if r <= self.prob_slip:

            shifts[z] = self.__random_offset()

        elif r <= self.prob_slip + self.prob_shift:

            offset = self.__random_offset()
            for zp in range(z, num_sections):
                shifts[zp] += offset

    logger.debug("misaligning sections with " + str(shifts))

    dims = 3
    bb_min = tuple(int(math.floor(transformation[d].min())) for d in range(dims))
    bb_max = tuple(int(math.ceil(transformation[d].max())) + 1 for d in range(dims))
    logger.debug("min/max of transformation: " + str(bb_min) + "/" + str(bb_max))

    for z in range(num_sections):
        transformation[1][z,:,:] += shifts[z][1]
        transformation[2][z,:,:] += shifts[z][2]

    bb_min = tuple(int(math.floor(transformation[d].min())) for d in range(dims))
    bb_max = tuple(int(math.ceil(transformation[d].max())) + 1 for d in range(dims))
    logger.debug("min/max of transformation after misalignment: " + str(bb_min) + "/" + str(bb_max))

作者使用了ElasticAugment([4,40,40], [0,2,2], [0,math.pi/2.0], prob_slip=0.05,prob_shift=0.05,max_misalign=25)
```
class gunpowder.ElasticAugment(control_point_spacing, jitter_sigma, rotation_interval, prob_slip=0, prob_shift=0, max_misalign=0, subsample=1)
```
- reshape array data into (channels,) + spatial dims
- 首先 **create_identity_transformation**
    - create_identity_transformation来自funkey的另一个repo augment，这个repo是之前的augment的简单方法，create_identity_transformation,从这里开始图像的channel会变成3,主要用np.meshgrid增加两个通道
- 其次 如果sum(jitter_sigma) > 0: **create_elastic_transformation**
```
augment.create_elastic_transformation(
                    target_shape,
                    self.control_point_spacing,
                    self.jitter_sigma,
                    subsample=self.subsample)
```
    -先得到control_point_offsets，再插值做upscale，依然生成3 channel图片
- 然后，由rotation_interval [0,math.pi/2.0], rotation_start = rotation_interval[0], rotation_max_amount = rotation_interval[1] - rotation_interval[0]

    rotation = random.random()*self.rotation_max_amount + self.rotation_start(在作者设置中是一个小于90度的值)
    
    如果rotation>0,做**create_rotation_transformation**
    
以上步骤都是之前的augment repo中实现的方法，作者直接在gunpowder中使用了

- 然后，如果prob_slip + prob_shift > 0，做**__misalign(transformation)**
    
    根据阈值做逐层的随机shift

- 最后，如果subsample >1（作者使用的是1），在做elastic augmentation之前会做subsample，加快elasticaugment的计算速度，做完augmentation再upscale

    Instead of creating an elastic transformation on the full
    resolution, create one subsampled by the given factor, and linearly
    interpolate to obtain the full resolution transformation. This can
    significantly speed up this node, at the expense of having visible
    piecewise linear deformations for large factors. Usually, a factor
    of 4 can savely by used without noticable changes. However, the
    default is 1 (i.e., no subsampling).

# DefectAugment
class gunpowder.DefectAugment(intensities, prob_missing=0.05, prob_low_contrast=0.05, prob_artifact=0.0, prob_deform=0.0, contrast_scale=0.1, artifact_source=None, artifacts=None, artifacts_mask=None, deformation_strength=20, axis=0)[source]

Augment intensity arrays section-wise with artifacts like missing sections, low-contrast sections, by blending in artifacts drawn from a separate source, or by deforming a section.

- intensities (ArrayKey) – The key of the array of intensities to modify.
- prob_missing (float) –
- prob_low_contrast (float) –
- prob_artifact (float) –
- prob_deform (float) – Probabilities of having a missing section, low-contrast section, an artifact (see param artifact_source) or a deformed slice. The sum should not exceed 1. Values in missing sections will be set to 0.
contrast_scale (float, optional) – By how much to scale the intensities for a low-contrast section, used if prob_low_contrast > 0.
- (class (artifact_source) – BatchProvider, optional):A gunpowder batch provider that delivers intensities (via ArrayKey artifacts) and an alpha mask (via ArrayKey artifacts_mask), used if prob_artifact > 0.

- artifacts (ArrayKey, optional) – The key to query artifact_source for to get the intensities of the artifacts.
- artifacts_mask (ArrayKey, optional) – The key to query artifact_source for to get the alpha mask of the artifacts to blend them with intensities.
- deformation_strength (int, optional) – Strength of the slice deformation in voxels, used if prob_deform > 0. The deformation models a fold by shifting the section contents towards a randomly oriented line in the section. The line itself will be drawn with a value of 0.
- axis (int, optional) – Along which axis sections are cut.


从一个专门的artifect文件读取数据并做defectaugment

数据如下，包括defects文件，raw和mask文件，并且在内部也做了一遍augmentation
```
artifact_source = (
        Hdf5Source(
            'sample_ABC_padded_20160501.defects.hdf',
            datasets = {
                ArrayKeys.RAW: 'defect_sections/raw',
                ArrayKeys.ALPHA_MASK: 'defect_sections/mask',
            }
        ) +
        RandomLocation(min_masked=0.05, mask_array_key=ArrayKeys.ALPHA_MASK) +
        Normalize() +
        IntensityAugment(0.9, 1.1, -0.1, 0.1, z_section_wise=True) +
        ElasticAugment([4,40,40], [0,2,2], [0,math.pi/2.0]) +
        SimpleAugment(transpose_only_xy=True)
    )
```
- 作者设置的阈值：
```
DefectAugment(
            prob_missing=0.03,
            prob_low_contrast=0.01,
            prob_artifact=0.03,
            artifact_source=artifact_source,
            contrast_scale=0.1)
```
- 分别获得missing，low_contrast,artifact,deform的值
    prob_missing_threshold = self.prob_missing   (0.03=0.03)
    prob_low_contrast_threshold = prob_missing_threshold + self.prob_low_contrast  (0.04=0.03+0.01)
    prob_artifact_threshold = prob_low_contrast_threshold + self.prob_artifact    (0.07=0.03+0.04)
    prob_deform_slice = prob_artifact_threshold + self.prob_deform   (0.07=0.07+0)
- 对每一个slice，产生一个0-1的随机数r，如果：
    - r < prob_missing_threshold:  'zero_out'
        - 什么也不做，直接输出
    - elif r < prob_low_contrast_threshold:  'lower_contrast'
        - mean = section.mean(), section -= mean, section *= self.contrast_scale, section += mean
    - elif r < prob_artifact_threshold:'artifact'
        - raw.data[section_selector] = section*(1.0 - artifact_alpha) + artifact_raw*artifact_alpha
        - artifact_alpha, artifact_raw来自artifact_source的mask和raw， 相当于做了一个Alpha Blending
    - elif r < prob_deform_slice: 'deformed_slice' 如果是deformed slice, 需要bigger upstream roi for deformed slice
        ```
        section = raw.data[section_selector].squeeze()
        interpolation = 3 if self.spec[self.intensities].interpolatable else 0
        flow_x, flow_y, line_mask = self.deform_slice_transformations[c]
        shape = section.shape
        #做双线性插值
        section = map_coordinates(
            section, (flow_y, flow_x), mode='constant', order=interpolation
        ).reshape(shape)
        section = np.clip(section, 0., 1.)
        section[line_mask] = 0
        raw.data[section_selector] = section
        ```